In [19]:
import pandas as pd
clicks_train = pd.read_csv('data/clicks_train.csv')
documents_categories = pd.read_csv('data/documents_categories.csv')
documents_meta = pd.read_csv('data/documents_meta.csv')
documents_entities = pd.read_csv('data/documents_entities.csv')
#page_view = pd.read_csv('data/page_views_sample.csv')
documents_topic = pd.read_csv('data/documents_topics.csv')
events = pd.read_csv('data/events.csv')

/Users/pranjor/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
pg_sample = page_view.sample(n=500000)

In [8]:
merge_df = pg_sample.merge(documents_categories, how='left', on='document_id')
merge_df = merge_df.merge(documents_entities, how='left', on='document_id', suffixes=('_cat', '_ent'))
merge_df = merge_df.merge(documents_meta,  how='left', on='document_id')

documents_topic = documents_topic.rename(columns = {'confidence_level':'confidence_level_top'})
merge_df = merge_df.merge(documents_topic,how='left', on='document_id')

In [4]:
#merge_df = merge_df.merge(events[['display_id','document_id']], how='left', on='document_id')

In [11]:
events[['display_id','document_id']].shape

(23120126, 2)

In [2]:
import pandas as pd
page_view = pd.read_csv('data/page_views_sample.csv')
page_view.head()

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1
3,8205775c5387f9,120,44196592,1,IN>16,2
4,9cb0ccd8458371,120,65817371,1,US>CA>807,2


In [20]:
import dask.dataframe as dd

# Read in the csv files.

dt = events.dtypes.to_dict()
df1 = dd.read_csv('data/clicks_train.csv')
df2 = dd.read_csv('data/events.csv', dtype={'uuid': object, 'platform': object, 'geo_location': object})
df3 = dd.read_csv('data/promoted_content.csv')

df4 = dd.read_csv('data/documents_categories.csv')
df4 = df4.groupby('document_id').max()
df4 = df4.reset_index()

df5 = dd.read_csv('data/documents_entities.csv', dtype={'entity_id': object})
df5 = df5.groupby('document_id').max()
df5 = df5.reset_index()

df6 = dd.read_csv('data/documents_meta.csv', dtype={'publish_time': object})

df7 = dd.read_csv('data/documents_topics.csv')
df7 = df7.groupby('document_id').max()
df7 = df7.reset_index()

In [38]:
import numpy as np
display_id = clicks_train.display_id.unique()
random_id = np.random.choice(display_id, 10000) 

In [39]:
new_sample = df1[df1.display_id.isin(random_id)]

In [4]:
random_sample = clicks_train[clicks_train.display_id.isin(random_id)]

In [40]:
df = dd.merge(new_sample, df2, how='left', on=['display_id'])
df = dd.merge(df, df3, how='left', on=['ad_id'], suffixes=('_event', '_prom'))
df = dd.merge(df, df4, how='left', left_on= 'document_id_event', right_on='document_id', left_index=True)
# df = dd.merge(df, df5, how='left', left_on= 'document_id_event', right_on='document_id', left_index=True)
# df = dd.merge(df, df6, how='left', left_on= 'document_id_event', right_on='document_id', left_index=True)
df = dd.merge(df, df7, how='left', on=['document_id'])

In [28]:
df.head()

,display_id,ad_id,clicked,uuid,document_id_event,timestamp,platform,geo_location,document_id_prom,campaign_id,advertiser_id,document_id,category_id,confidence_level_x,topic_id,confidence_level_y
0,8562432,51384.0,0.0,93e3762a8e0312,2116651.0,579816437.0,1,US>MI>551,969326.0,6901.0,1438.0,2116651.0,1513.0,0.750735,258.0,0.210745
1,8562432,154575.0,1.0,93e3762a8e0312,2116651.0,579816437.0,1,US>MI>551,1381560.0,19670.0,1378.0,2116651.0,1513.0,0.750735,258.0,0.210745
2,8562432,160674.0,0.0,93e3762a8e0312,2116651.0,579816437.0,1,US>MI>551,1218152.0,19027.0,2926.0,2116651.0,1513.0,0.750735,258.0,0.210745
3,8562432,179047.0,0.0,93e3762a8e0312,2116651.0,579816437.0,1,US>MI>551,1351158.0,21726.0,3190.0,2116651.0,1513.0,0.750735,258.0,0.210745
4,8581536,9275.0,0.0,bdbbd7aed302a5,1765644.0,580726924.0,2,US>OK>671,594144.0,77.0,53.0,1765644.0,1702.0,0.920000,102.0,0.073407


In [152]:
a = documents_categories.groupby('document_id')[['document_id', '']].max()
a = a.reset_index()

In [41]:
df.to_csv('merge-*.csv', index=False)

In [42]:
import glob

allFiles = glob.glob("merge-*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,)
    list_.append(df)
frame = pd.concat(list_)

In [44]:
frame.to_csv('merged.csv')

In [43]:
frame.shape

(51658, 16)

In [45]:
ev_entity = pd.merge(event[['document_id']],entity,on='document_id',how='left')
entity_list = ev_entity.groupby('entity_id').count().sort_values(by='document_id',ascending=False).iloc[:10].index.values.tolist()

def entity_function(e):
   if e in entity_list:
       return e
   else:
       return 'MinorityEntity'
ev_entity = ev_entity.drop_duplicates('document_id')
ev_entity['entity_id'] = ev_entity['entity_id'].apply(entity_function)
ev_entity.shape

,document_id,source_id,publisher_id,publish_time
0,1595802,1.0,603.0,2016-06-05 00:00:00
1,1524246,1.0,603.0,2016-05-26 11:00:00
2,1617787,1.0,603.0,2016-05-27 00:00:00
3,1615583,1.0,603.0,2016-06-07 00:00:00
4,1615460,1.0,603.0,2016-06-20 00:00:00
